In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
from vmas import make_env
from vmas.simulator.core import Agent
from vmas.simulator.scenario import BaseScenario
from typing import Union
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML, display as ipython_display

class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class DQNAgent:
    def __init__(self, state_dim, action_dim, device, lr=1e-3, gamma=0.99, epsilon_start=1.0, epsilon_end=0.01, epsilon_decay=0.995):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.device = device
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay

        self.q_network = QNetwork(state_dim, action_dim).to(device)
        self.target_network = QNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.memory = deque(maxlen=10000)
        self.batch_size = 64

        self.update_target_network()

    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

    def get_action(self, state, agent_silent):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self.action_dim if not agent_silent else 1)
        else:
            state = torch.FloatTensor(state).to(self.device)
            with torch.no_grad():
                q_values = self.q_network(state)
            return q_values.argmax().item() if not agent_silent else 0  # Default to 0 if silent

    def add_to_memory(self, transition):
        self.memory.append(transition)

    def train(self):
        if len(self.memory) < self.batch_size:
            return
        
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(self.device)

        current_q_values = self.q_network(states).gather(1, actions)
        with torch.no_grad():
            next_q_values = self.target_network(next_states).max(1)[0].unsqueeze(1)
            target_q_values = rewards + (1 - dones) * self.gamma * next_q_values

        loss = nn.functional.mse_loss(current_q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.epsilon = max(self.epsilon_end, self.epsilon_decay * self.epsilon)

class VMASEnvRunner:
    def __init__(
        self,
        render: bool,
        num_envs: int,
        num_episodes: int,
        max_steps_per_episode: int,
        device: str,
        scenario: Union[str, BaseScenario],
        continuous_actions: bool,
        random_action: bool,
        **kwargs
    ):
        self.render = render
        self.num_envs = num_envs
        self.num_episodes = num_episodes
        self.max_steps_per_episode = max_steps_per_episode
        self.device = device
        self.scenario = scenario
        self.continuous_actions = continuous_actions
        self.random_action = random_action
        self.kwargs = kwargs
        self.frame_list = []  # Initialize frame list

        # Initialize DQN agent
        self.dqn_agent = DQNAgent(state_dim=4, action_dim=3, device=device)  # Adjust state_dim and action_dim as needed

    def get_continuous_action(self, state, agent_silent):
        return self.dqn_agent.get_action(state, agent_silent)

    def get_discrete_action(self):
        pass

    def _get_deterministic_action(self, agent: Agent, env, obs, agent_id):
        if self.continuous_actions:
            # Use DQN to get action
            state = obs[agent_id]  # Obtain the state from the environment
            action_idx = self.get_continuous_action(state, agent.silent)
            if agent.silent:
                action = torch.tensor([[-1, 0.5]], device=self.device)  # Default silent action
            else:
                action = torch.tensor([action_idx], device=self.device)  # Convert action index to tensor
        else:
            action = self.get_discrete_action()
            if(agent.silent):
                action = torch.tensor([[8]], device=env.device) # range action is [0,8] of a single value
            else:
                action = torch.tensor([[8]], device=env.device) # range action is [0,8] of a single value
        return action.clone()

    def generate_gif(self, scenario_name):
        fps = 30
        clip = ImageSequenceClip(self.frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)

        # Return the HTML tag to display the GIF
        return HTML(f'<img src="{scenario_name}.gif">')

    def run_vmas_env(self):
        scenario_name = self.scenario if isinstance(self.scenario, str) else self.scenario.__class__.__name__

        env = make_env(
            scenario=self.scenario,
            num_envs=self.num_envs,
            device=self.device,
            continuous_actions=self.continuous_actions,
            seed=0,
            **self.kwargs
        )

        init_time = time.time()
        total_steps = 0
        
        for e in range(self.num_episodes):  # Loop over episodes
            print(f"Episode {e}")
            obs = env.reset()  # Reset environment at the start of each episode
            done = [False] * self.num_envs
            step = 0
            while not all(done) and step < self.max_steps_per_episode:  # Loop over steps within an episode
                step += 1
                total_steps += 1
                print(f"Step {step} of Episode {e}")

                actions = []
                for i, agent in enumerate(env.agents):
                    if not self.random_action:
                        action = self._get_deterministic_action(agent, env, obs, i)
                    else:
                        action = env.get_random_action(agent)

                    print(f"action agent {i}: {action}")

                    actions.append(action)

                next_obs, rews, dones, info = env.step(actions)
                done = [done or d for done, d in zip(done, dones)]

                # Add transition to DQN memory and train
                self.dqn_agent.add_to_memory((obs, actions, rews, next_obs, dones))
                self.dqn_agent.train()

                obs = next_obs

                print(f"obs agent after step {step}: {obs}")
                print(f"rews agent: {rews}")

                if self.render:
                    frame = env.render(
                        mode="rgb_array",
                        agent_index_focus=None,
                    )
                    self.frame_list.append(frame)  # Append frame to the list

        total_time = time.time() - init_time
        print(
            f"It took: {total_time}s for {total_steps} steps across {self.num_episodes} episodes of {self.num_envs} parallel environments on device {self.device} "
            f"for {scenario_name} scenario."
        )

if __name__ == "__main__":
    scenario_name = "navigation_comm"

    env_runner = VMASEnvRunner(
        render=True,
        num_envs=1,
        num_episodes=1,
        max_steps_per_episode=10,
        device="cuda",
        scenario=scenario_name,
        continuous_actions=True,
        random_action=False,
        # Environment specific variables
        n_agents=3,
    )
    # Run the VMAS environment
    env_runner.run_vmas_env()

    # Generate and display the GIF
    ipython_display(env_runner.generate_gif(scenario_name))


Episode 0
Step 1 of Episode 0
action agent 0: tensor([1], device='cuda:0')
action agent 1: tensor([1], device='cuda:0')
action agent 2: tensor([2], device='cuda:0')


AssertionError: Action for agent agent_0 has shape 1, but should have shape 4